In [1]:
import os
from fastapi import FastAPI
from fastapi.encoders import jsonable_encoder
import pandas as pd
import json
from services.ResumeInfoExtraction import ResumeInfoExtraction
from services.JobInfoExtraction import JobInfoExtraction
from source.schemas.resumeextracted import ResumeExtractedModel
from source.schemas.jobextracted import JobExtractedModel
import ast
from pypdf import PdfReader
import warnings
warnings.filterwarnings("ignore")

def get_resumes(directory):
    
    def extract_pdf(path):
        reader = PdfReader(path)
        number_of_pages = len(reader.pages)
        text = ""
        for i in range(number_of_pages):
            page = reader.pages[i]
            text += page.extract_text()
        return text
    
    dic = {}
    
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        if os.path.isfile(file_path) and filename.endswith(".pdf"):
            name = filename.strip(".pdf")
            resume_text = extract_pdf(file_path)
            dic[name] = [resume_text]
    
    df = pd.DataFrame(dic).T
    df.reset_index(inplace=True)
    df.rename(columns={"index": "name", 0:"raw"}, inplace=True)
    
    return df

def transform_dataframe_to_json(dataframe):

    # transforms the dataframe into json
    result = dataframe.to_json(orient="records")
    parsed = json.loads(result)
    json_data = json.dumps(parsed, indent=4)

    return json_data


def extraction(resume):
    degrees_patterns_path = 'Resources/data/degrees.jsonl'
    majors_patterns_path = 'Resources/data/majors.jsonl'
    skills_patterns_path = 'Resources/data/skills.jsonl'
    jobs = resume
    names = transform_dataframe_to_json(jobs[["name"]])
    job_extraction = ResumeInfoExtraction(skills_patterns_path, majors_patterns_path, degrees_patterns_path, jobs, names)
    jobs = job_extraction.extract_entities(jobs)
    for i, row in jobs.iterrows():
        name = row["name"]
        degrees = jobs.loc[i, 'Degrees']
        maximum_degree_level = jobs.loc[i, 'Maximum degree level']
        acceptable_majors = jobs.loc[i, 'Acceptable majors']
        skills = jobs.loc[i, 'Skills']
        

        job_extracted = ResumeExtractedModel(maximum_degree_level=maximum_degree_level if maximum_degree_level else '',
                                          acceptable_majors=acceptable_majors if acceptable_majors else [],
                                          skills=skills if skills else [],
                                          name=name if name else '',
                                          degrees=degrees if degrees else [])
        job_extracted = jsonable_encoder(job_extracted)
    jobs_json = transform_dataframe_to_json(jobs)
    
    return jobs_json

if __name__ == "__main__":
    df = get_resumes("resumes")
    res = extraction(df)
    df = pd.read_json(res)



c:\Users\victo\anaconda3\envs\workproject\Lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)


In [2]:
df

,name,raw,Degrees,Maximum degree level,Acceptable majors,Skills
0,hanna_pedersen,EDUCATION Nova School of Business an...,"[MS-LEVEL, BS-LEVEL]",MS-LEVEL,"[data analysis, artificial intelligence, progr...","[business, analytics, business administration,..."
1,irene_abbateli,"\n \n \nIRENE ABBATELLI \nRome, Italy \n(+39)...","[MS-LEVEL, BS-LEVEL]",MS-LEVEL,[data analysis],"[business, analytics, data analysis, machine l..."
2,victor_bjorsvik,"VICTOR BJORSVIK \n \nLisbon, Portugal | +47 ...","[BS-LEVEL, MS-LEVEL]",MS-LEVEL,"[programming, data analysis, data science, web...","[accounting, big data, communications, busines..."


In [3]:
%%bash
ls job_descriptions/

description.txt


In [4]:
with open('job_descriptions/description.txt', 'r') as file:
    job_description = file.read()

job_description = [job_description]
df2 = pd.DataFrame(job_description, columns=["raw"])
df2

,raw
0,Data Scientist\nLisbon\n\n\n\nApply on employe...


In [5]:
def extraction(resume):
    degrees_patterns_path = 'Resources/data/degrees.jsonl'
    majors_patterns_path = 'Resources/data/majors.jsonl'
    skills_patterns_path = 'Resources/data/skills.jsonl'
    jobs = resume
    job_extraction = JobInfoExtraction(skills_patterns_path, majors_patterns_path, degrees_patterns_path, jobs)
    jobs = job_extraction.extract_entities(jobs)
    for i, row in jobs.iterrows():
        minimum_degree_level = jobs['Minimum degree level'][i]
        acceptable_majors = jobs['Acceptable majors'][i]
        skills = jobs['Skills'][i]

        job_extracted = JobExtractedModel(minimum_degree_level=minimum_degree_level if minimum_degree_level else '',
                                          acceptable_majors=acceptable_majors if acceptable_majors else [],
                                          skills=skills if skills else [])
        job_extracted = jsonable_encoder(job_extracted)
        # new_job_extracted = database.get_collection("jobsextracted").insert_one(job_extracted)
    jobs_json = transform_dataframe_to_json(jobs)
    return jobs_json

In [6]:
res = extraction(df2)
df2 = pd.read_json(res)
df2

,raw,Minimum degree level,Acceptable majors,Skills
0,Data Scientist\nLisbon\n\n\n\nApply on employe...,BS-LEVEL,"[data analysis, information technology, comput...","[finance, business, visualization, design, mac..."


In [8]:
df2.Skills[0]

['finance',
 'business',
 'visualization',
 'design',
 'machine learning',
 'modelling',
 'data analysis',
 'python',
 'marketing',
 'operations research',
 'database',
 'data quality',
 'analytics',
 'agile project management',
 'training model',
 'computer engineering',
 'relational database',
 'oracle',
 'collaboration',
 'communications',
 'data science']

In [10]:
df.Skills[2]

['accounting',
 'big data',
 'communications',
 'business',
 'analytics',
 'finance',
 'data science',
 'modelling',
 'business administration',
 'python',
 'logistic regression',
 'web development',
 'flask',
 'html',
 'css',
 'javascript',
 'testing',
 'training model',
 'pandas',
 'sql',
 'apache spark',
 'machine learning',
 'tensorflow',
 'keras',
 'c',
 'r',
 'tableau']